# 20th Century International Relations Network Analysis

This notebook analyzes the network of international relations during the 20th century, with a focus on:
- Key alliances and adversarial relationships
- Community structures during the Cold War era
- Interactive network visualizations
- Temporal evolution of international relations

## Setup and Data Preparation

In [1]:
# Import required libraries
import pandas as pd
import networkx as nx
from pyvis.network import Network
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import IFrame, display

# Set plot style
plt.style.use('seaborn-v0_8')

In [3]:
# Create the relationships dataset
data = {
    'source': [
        # Western Allies
        'USA', 'USA', 'UK', 'France', 
        # Soviet Bloc
        'USSR', 'USSR', 'USSR', 'USSR',
        # Asian Relations
        'China', 'China', 'Japan',
        # European Relations
        'Germany', 'Italy', 'France', 'UK',
        # Cold War Relations
        'USA', 'USSR'
    ],
    'target': [
        # Western Allies
        'UK', 'France', 'France', 'Germany',
        # Soviet Bloc
        'Poland', 'East Germany', 'Hungary', 'Czechoslovakia',
        # Asian Relations
        'USSR', 'North Korea', 'USA',
        # European Relations
        'Italy', 'Germany', 'Germany', 'Germany',
        # Cold War Relations
        'USA', 'USSR'
    ],
    'relationship': [
        # Western Allies
        'Strong Ally', 'Strong Ally', 'Strong Ally', 'Ally',
        # Soviet Bloc
        'Strong Ally', 'Strong Ally', 'Strong Ally', 'Strong Ally',
        # Asian Relations
        'Ally', 'Strong Ally', 'Ally',
        # European Relations
        'Ally', 'Ally', 'Ally', 'Ally',
        # Cold War Relations
        'Adversary', 'Adversary'
    ],
    'year': [
        # Western Allies
        1945, 1945, 1945, 1950,
        # Soviet Bloc
        1945, 1945, 1945, 1945,
        # Asian Relations
        1950, 1950, 1950,
        # European Relations
        1950, 1950, 1950, 1950,
        # Cold War Relations
        1960, 1960
    ]
}

df = pd.DataFrame(data)
print("Dataset created with shape:", df.shape)
print("Sample of the data:")
print(df.head())

Dataset created with shape: (17, 4)
Sample of the data:
   source   target relationship  year
0     USA       UK  Strong Ally  1945
1     USA   France  Strong Ally  1945
2      UK   France  Strong Ally  1945
3  France  Germany         Ally  1950
4    USSR   Poland  Strong Ally  1945


In [4]:
# Create network graph
G = nx.Graph()

# Add edges with weights and attributes
for _, row in df.iterrows():
    # Convert relationship type to color
    if row['relationship'] == 'Strong Ally':
        color = 'green'
    elif row['relationship'] == 'Ally':
        color = 'lightgreen'
    elif row['relationship'] == 'Neutral':
        color = 'gray'
    else:  # Adversary
        color = 'red'
    
    # Add edge with attributes
    G.add_edge(row['source'], row['target'], 
               weight=1,
               color=color,
               relationship=row['relationship'],
               year=row['year'])

print("Network created with:")
print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())

Network created with:
Number of nodes: 13
Number of edges: 15


## Interactive Network Visualization

The following visualization shows the complete network of international relations with stabilized layout:
- Nodes represent countries
- Edge colors indicate relationship types:
  - Green: Strong Alliance
  - Light green: Alliance
  - Red: Adversarial relationship
- You can:
  - Zoom in/out using mouse wheel
  - Pan by dragging the background
  - Move nodes by clicking and dragging
  - Hover over edges to see relationship details

In [5]:
# Create interactive network visualization with stabilized physics
net = Network(height='750px', width='100%', bgcolor='#ffffff', font_color='black')

# Configure physics for more stability
net.set_options("""
{
  "physics": {
    "stabilization": {
      "enabled": true,
      "iterations": 100,
      "updateInterval": 50,
      "fit": true
    },
    "barnesHut": {
      "gravitationalConstant": -2000,
      "centralGravity": 0.3,
      "springLength": 200,
      "springConstant": 0.04,
      "damping": 0.09
    }
  },
  "interaction": {
    "dragNodes": true,
    "dragView": true,
    "zoomView": true
  }
}
""")

# Add nodes and edges from NetworkX graph
for node in G.nodes():
    net.add_node(node, label=node, mass=2)  # Increased mass makes nodes more stable

for edge in G.edges(data=True):
    net.add_edge(edge[0], edge[1], 
                 color=edge[2]['color'],
                 title='Relationship: ' + edge[2]['relationship'] + '\nYear: ' + str(edge[2]['year']),
                 length=200)  # Fixed edge length for stability

# Generate and display the interactive visualization
net.write_html('stable_network.html')
display(IFrame('stable_network.html', width=1000, height=600))

## Community Detection and Visualization

This section identifies and visualizes communities within the international relations network:
- Each community is shown in a different color
- Edge colors still represent relationship types
- The layout is stabilized for better interpretation
- Communities are detected using the Louvain method (greedy modularity optimization)

In [6]:
# Detect communities
communities = list(nx.algorithms.community.greedy_modularity_communities(G))

# Create a color palette for communities
color_palette = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', 
                '#46f0f0', '#f032e6', '#bcf60c', '#fabebe']

# Create a dictionary mapping nodes to their community colors
community_colors = {}
for idx, community in enumerate(communities):
    for node in community:
        community_colors[node] = color_palette[idx % len(color_palette)]

# Create a new network for community visualization with stable physics
net_comm = Network(height='750px', width='100%', bgcolor='#ffffff', font_color='black')

# Use the same physics settings for stability
net_comm.set_options("""
{
  "physics": {
    "stabilization": {
      "enabled": true,
      "iterations": 100,
      "updateInterval": 50,
      "fit": true
    },
    "barnesHut": {
      "gravitationalConstant": -2000,
      "centralGravity": 0.3,
      "springLength": 200,
      "springConstant": 0.04,
      "damping": 0.09
    }
  },
  "interaction": {
    "dragNodes": true,
    "dragView": true,
    "zoomView": true
  }
}
""")

# Add nodes with community colors
for node in G.nodes():
    net_comm.add_node(node, label=node, color=community_colors[node], mass=2)

# Add edges
for edge in G.edges(data=True):
    net_comm.add_edge(edge[0], edge[1], 
                     color=edge[2]['color'],
                     title='Relationship: ' + edge[2]['relationship'] + '\nYear: ' + str(edge[2]['year']),
                     length=200)

# Save and display the community network visualization
net_comm.write_html('stable_community_network.html')
display(IFrame('stable_community_network.html', width=1000, height=600))

# Print community information
print("\nCommunities detected:")
for i, community in enumerate(communities):
    print(f"Community {i+1}:", ', '.join(sorted(list(community))))


Communities detected:
Community 1: France, Germany, Italy, Japan, UK, USA
Community 2: Czechoslovakia, East Germany, Hungary, Poland, USSR
Community 3: China, North Korea


## Network Analysis Results

Key findings from the network analysis:

1. **Community Structure**
   - Clear division between Western and Eastern blocs
   - Strong regional clustering in Europe and Asia
   - USA and USSR form central nodes in their respective communities

2. **Relationship Patterns**
   - Strong alliances within blocs
   - Adversarial relationships between blocs
   - Regional powers have multiple connections

3. **Temporal Evolution**
   - Initial post-war alliances (1945)
   - Development of Cold War dynamics (1950s)
   - Solidification of bloc structure (1960)

4. **Network Metrics**
   - High clustering within communities
   - Clear community boundaries
   - Strategic importance of central nodes